In [175]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from path import Path
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [176]:
from numpy.random import seed
seed(1)
from tensorflow import random
random.set_seed(2)

In [177]:
file_path = Path("../../data/cleandata/1hot_merged_post_ICO.csv")
df = pd.read_csv(file_path)

In [183]:
df.fillna(0, inplace=True)

In [207]:
X = df.drop(columns = ['Ticker','Name', 'success'])
y = df["success"].values

In [208]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
X_scaler = StandardScaler()
X_scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [209]:
results = []
activations = ['elu', 'relu', 'softmax', 'selu', 'softplus', 'softsign', 'tanh', 'sigmoid', 'hard_sigmoid', 'exponential', 'linear']
optimizers = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
loss_functions = ['mean_squared_error',
                  'mean_absolute_error',
                 'mean_squared_logarithmic_error',
                 'squared_hinge',
                 'hinge',
                 'categorical_hinge',
                 'logcosh',
                 'huber_loss',
                 'binary_crossentropy',
                 'kullback_leibler_divergence',
                 'poisson',
                 ]

In [ ]:
for activator in activations:
    for activator2 in activations:
        for final_activator in activations:
            for optimizer in optimizers:    
                for loss_function in loss_functions:

                    nn = Sequential()

                    nn.add(Dense(units=8, input_dim=95, activation=activator))

                    nn.add(Dense(units=10, activation=activator2))

                    # Output layer
                    nn.add(Dense(units=1, activation=final_activator))

                    # Compile the model
                    nn.compile(loss=loss_function, optimizer=optimizer, metrics=["mse"])

                    # Fit the model
                    model = nn.fit(X_train_scaled, y_train, epochs=10, verbose=0)

                    predicted = nn.predict(X_test_scaled)

                    model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)

                    result = {
                        'combo':[activator, activator2, final_activator, loss_function, optimizer],
                        'loss' : model_loss,
                        'accuracy' : model_accuracy,
                    }

                    results.append(result)


135/1 - 0s - loss: 0.0979 - mse: 0.0952
135/1 - 0s - loss: 0.1356 - mse: 0.0864
135/1 - 0s - loss: 0.0080 - mse: 0.1820
135/1 - 0s - loss: 0.0948 - mse: 0.6809
135/1 - 0s - loss: 0.2172 - mse: 0.7295
135/1 - 0s - loss: 0.2154 - mse: 0.7552
135/1 - 0s - loss: 0.0661 - mse: 0.1975
135/1 - 0s - loss: 0.0401 - mse: 0.0890
135/1 - 0s - loss: 0.2360 - mse: 0.3025
135/1 - 0s - loss: 0.1240 - mse: 0.3102
135/1 - 0s - loss: nan - mse: 0.7589
135/1 - 0s - loss: 0.0366 - mse: 0.0451
135/1 - 0s - loss: 0.0631 - mse: 0.0345
135/1 - 0s - loss: 0.0066 - mse: 0.2010
135/1 - 0s - loss: 0.1085 - mse: 0.6090
135/1 - 0s - loss: 0.3115 - mse: 0.6557
135/1 - 0s - loss: 0.0987 - mse: 0.9097
135/1 - 0s - loss: 0.0099 - mse: 0.0328
135/1 - 0s - loss: 0.1751 - mse: 0.2169
135/1 - 0s - loss: 0.3742 - mse: 0.4293
135/1 - 0s - loss: 0.1221 - mse: 0.4687
135/1 - 0s - loss: nan - mse: 0.6875
135/1 - 0s - loss: 0.4006 - mse: 0.5462
135/1 - 0s - loss: 0.5992 - mse: 0.3808
135/1 - 0s - loss: 0.0745 - mse: 0.1212
135/1 

In [ ]:
x = pd.DataFrame(results)

In [ ]:
fsd = x.sort_values('loss', ascending=True)

In [ ]:
fsd[:5]

In [ ]:
rfgc  = x.sort_values('accuracy', ascending=False)
rfgc[:5]

In [220]:
nn = Sequential()

nn.add(Dense(units=8, input_dim=95, activation='exponential'))

nn.add(Dense(units=10, activation='elu'))

nn.add(Dense(units=10, activation='softplus'))

# Output layer
nn.add(Dense(units=1, activation="sigmoid"))

# Compile the model
nn.compile(loss='kullback_leibler_divergence', optimizer='SGD', metrics=["mse"])

# Fit the model
model = nn.fit(X_train_scaled, y_train, epochs=1000, verbose=0)

predicted = nn.predict(X_test_scaled)

model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

135/1 - 0s - loss: 4.1882e-05 - mse: 0.9485
Normal Neural Network - Loss: 8.5374533067874e-05, Accuracy: 0.9484952688217163


In [212]:
nn_json = nn.to_json()

file_path = Path("./christian_nn3_10k_new_df.json")

with open(file_path, "w") as json_file:
    json_file.write(nn_json)

# Save weights
file_path = Path("./christian_nn3_10k_new_df.h5")
nn.save_weights(file_path)

In [213]:
file_path2 = Path("../../data/cleandata/1hot_merged_360_ICO.csv")
ico_360 = pd.read_csv(file_path2)

In [215]:
X = ico_360.drop(columns = ['Ticker','Name', 'success'])
X_scaler = StandardScaler()
X_scaler.fit(X)
X_scaled = X_scaler.transform(X)
predictions = nn.predict(X_scaled)

In [216]:
score = predictions * 100
score

array([[ 94.39224 ],
       [ 94.09292 ],
       [ 91.83928 ],
       [ 99.99766 ],
       [ 94.649506],
       [ 94.097725],
       [ 83.99434 ],
       [ 99.20571 ],
       [ 99.718   ],
       [ 98.03781 ],
       [ 93.24653 ],
       [ 99.45988 ],
       [ 99.86925 ],
       [ 99.99633 ],
       [ 99.520256],
       [ 98.64129 ],
       [ 95.3054  ],
       [ 95.635864],
       [ 70.5647  ],
       [ 94.068054],
       [ 84.67967 ],
       [ 85.269035],
       [ 99.86279 ],
       [ 99.707375],
       [ 95.4961  ],
       [ 96.08528 ],
       [ 99.44078 ],
       [100.      ],
       [100.      ],
       [ 95.467316],
       [ 99.82647 ],
       [ 97.08205 ],
       [ 89.699005],
       [ 97.92468 ],
       [ 94.152275],
       [ 88.699974]], dtype=float32)

In [217]:
results = []
for s in score:
    results.append(*s)
results

[94.39224,
 94.09292,
 91.83928,
 99.99766,
 94.649506,
 94.097725,
 83.99434,
 99.20571,
 99.718,
 98.03781,
 93.24653,
 99.45988,
 99.86925,
 99.99633,
 99.520256,
 98.64129,
 95.3054,
 95.635864,
 70.5647,
 94.068054,
 84.67967,
 85.269035,
 99.86279,
 99.707375,
 95.4961,
 96.08528,
 99.44078,
 100.0,
 100.0,
 95.467316,
 99.82647,
 97.08205,
 89.699005,
 97.92468,
 94.152275,
 88.699974]

In [218]:
ico_360["Viability"] = results

In [219]:
ico_360.sort_values('Viability', ascending = False)

,Name,Ticker,Duration,pre_Duration,compound,negative,neutral,positive,no_of_posts,Longevity,...,platform_Litecoin,platform_NEM,platform_NEO,platform_Nxt,platform_Omni,platform_Own Blockchain,platform_Scrypt,platform_Stellar,platform_Waves,Viability
28,ICOCalendar.T...,ICT,27,0,0.867340,0.090800,0.030000,0.879400,5,321,...,0,0,0,0,0,0,0,0,0,100.000000
27,BowsCoin,BSC,13,13,0.576600,0.014000,0.005000,0.981000,1,249,...,0,0,0,0,0,0,0,0,0,100.000000
3,BQT,BQTX,181,0,-0.177867,0.006333,0.011333,0.982333,3,243,...,0,0,0,0,0,0,0,0,0,99.997658
13,COTI,COTI,10,0,-0.648600,0.026000,0.962000,0.011000,1,155,...,0,0,0,0,0,0,0,0,0,99.996330
12,Mindexcoin,MIC,14,9,-0.514596,0.060261,0.891826,0.047913,23,336,...,0,0,0,0,0,0,0,0,0,99.869247
22,GoldFund,GFUN,61,60,-0.690800,0.007000,0.020000,0.973000,1,289,...,1,0,0,0,0,0,0,0,0,99.862793
30,Dexter G,DXG,150,31,0.600258,0.091700,0.042770,0.865540,100,169,...,0,0,0,0,0,0,0,0,0,99.826469
8,HashBX,HBX,30,183,-0.285550,0.007000,0.011500,0.981000,2,341,...,0,0,0,0,0,0,0,0,0,99.718002
23,FirstCoin,FRST,19,61,0.903400,0.034000,0.028000,0.938000,1,312,...,0,0,0,0,0,0,0,0,0,99.707375
14,BonusCloud,BXC,59,26,0.127433,0.015000,0.013000,0.972000,3,222,...,0,0,0,0,0,0,0,0,0,99.520256
